内存映射文件使用操作系统虚拟内存直接访问文件上的数据，而不是通过常规的 I/O 方法。内存映射通常可以提高 I/O 性能，因为对于每次访问它不涉及单独的系统调用，也不会在缓冲池之间复制数据，而是内核和用户程序可以直接访问内存。
## 读

In [1]:
import mmap

with open('lorem.txt', 'r') as f:
    with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as m:
        print('First 10 bytes via read:', m.read(10))
        print('First 10 bytes via slice:', m[:10])
        print('2nd 10 bytes via read:', m.read(10))

First 10 bytes via read: b'Lorem ipsu'
First 10 bytes via slice: b'Lorem ipsu'
2nd 10 bytes via read: b'm dolor si'


使用 mmap() 创建一个内存映射文件。第一个参数是一个文件描述符，要么是来自 file 对象的 fileno() 方法， 要么来自 os.open()。  
第二个参数传入 mmap() 的参数是要去映射的文件内容的大小。如果值是 0 ，那么代表映射整个文件。  
可选的参数 access ，使用 ACCESS_READ 表示只读，ACCESS_WRITE 表示直接写（对内存的操作直接写入文件），或者 ACCESS_COPY 用于写时复制（内存分配不写入文件）。
## 写

In [4]:
import shutil
shutil.copyfile('lorem.txt', 'lorem_copy.txt')
word = b"consectetuer"
reversed = word[::-1]
print('looking for ', word)
print('replacing for ', reversed)
with open('lorem_copy.txt', 'r+') as f:
    with mmap.mmap(f.fileno(), 0) as m:
        print(f"Before:{m.readline().rstrip()}")
        m.seek(0)
        
        loc = m.find(word)
        m[loc:loc+len(word)] = reversed
        m.flush()
        
        m.seek(0)
        print(f"After:{m.readline().rstrip()}")
        
        f.seek(0)
        print(f"File:{f.readline().rstrip()}")

looking for  b'consectetuer'
replacing for  b'reutetcesnoc'
Before:b'Lorem ipsum dolor sit amet, consectetuer adipiscing elit.'
After:b'Lorem ipsum dolor sit amet, reutetcesnoc adipiscing elit.'
File:Lorem ipsum dolor sit amet, reutetcesnoc adipiscing elit.


为了设置一个内存映射文件去接受更新，要以追加模式 r+ （而不是 w）打开然后再进行映射
## 复制模式

In [5]:
shutil.copyfile('lorem.txt', 'lorem_copy.txt')
word = b"consectetuer"
reversed = word[::-1]
print('looking for ', word)
print('replacing for ', reversed)
with open('lorem_copy.txt', 'r+') as f:
    with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_COPY) as m:
        print(f"Before:{m.readline().rstrip()}")
        print(f"File Before:{f.readline().rstrip()}")
        m.seek(0)
        
        loc = m.find(word)
        m[loc:loc+len(word)] = reversed
        m.flush()
        
        m.seek(0)
        print(f"After:{m.readline().rstrip()}")
        
        f.seek(0)
        print(f"File After:{f.readline().rstrip()}")

looking for  b'consectetuer'
replacing for  b'reutetcesnoc'
Before:b'Lorem ipsum dolor sit amet, consectetuer adipiscing elit.'
File Before:Lorem ipsum dolor sit amet, consectetuer adipiscing elit.
After:b'Lorem ipsum dolor sit amet, reutetcesnoc adipiscing elit.'
File After:Lorem ipsum dolor sit amet, consectetuer adipiscing elit.


## 正则

In [6]:
import re

pattern = re.compile(rb'(\.\W+)?([^.]?nulla[^.]*?\.)', re.DOTALL | re.IGNORECASE | re.MULTILINE)
with open('lorem.txt', 'r') as f:
    with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as m:
        for match in pattern.findall(m):
            print(match[1].replace(b'\n', b' '))

b'Nulla facilisi.'
b'Nulla feugiat augue eleifend nulla.'
